In [1]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

np.random.seed(42)
X, y = torch.load('data/house_price.pkl')
train_data, test_data = X.iloc[:len(y)], X.iloc[len(y):]

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error, make_scorer

# baseline
scaler = StandardScaler()
svr = SVR()
pipeline = Pipeline([
    ('std', scaler),
    ('svr', svr)
])
cross_val_score(svr, train_data, y, scoring=make_scorer(mean_squared_log_error, greater_is_better=False), cv=5).mean()


-0.1599420981021455

In [ ]:
from ray.tune.sklearn import TuneSearchCV
from ray import tune
from sklearn.metrics import mean_absolute_percentage_error

params = {
    'svr__C': tune.loguniform(1e-1, 10),
    'svr__gamma': tune.loguniform(1e-2, 1)
    'svr__epsilon': tune.loguniform(1e-2, 1)
}
search = TuneSearchCV(
    pipeline,
    params,
    search_optimization="bayesian",
    n_trial=-1,
    max_iters=10,
    early_stopping=True,
    verbose=1,
    scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False),
    return_train_score=True,
    mode='max',
    time_budget_s=600,
    error_score=np.nan,
    use_gpu=True
)
search.fit(train_data, y)
print(search.best_params_)
print(search.best_score_)